# 付録 10.2: ツール使用

- [レッスン](#lesson)
- [演習](#exercises)
- [プレイグラウンド例](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を設定してください。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

# Override the MODEL_NAME variable in the IPython store to use Sonnet instead of the Haiku model
MODEL_NAME='anthropic.claude-3-sonnet-20240229-v1:0'
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

# Rewrittten to call Claude 3 Sonnet, which is generally better at tool use, and include stop_sequences
def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages,
        system=system_prompt,
        stop_sequences=stop_sequences
    )
    return message.content[0].text

---

## レッスン

最初は概念的に複雑に見えるかもしれませんが、ツール使用（関数呼び出しとも呼ばれる）は実際にはとても簡単です！ツール使用を実装するために必要なスキルはすべてすでに習得しており、実際には置換とプロンプトチェーンの組み合わせに過ぎません。

以前の置換演習では、プロンプトにテキストを置換しました。ツール使用では、ツールや関数の結果をプロンプトに置換します。Claudeは文字通りツールや関数を呼び出したりアクセスしたりすることはできません。代わりに、Claudeに以下のことをさせます：
1. 呼び出したいツール名と引数を出力する
2. ツールが呼び出される間、それ以上の応答生成を停止する
3. その後、追加されたツールの結果で再プロンプトする

関数呼び出しは、Claudeの機能を拡張し、Claudeがより複雑で複数ステップのタスクを処理できるようにするため便利です。
Claudeに提供できる関数の例：
- 電卓
- 単語カウンター
- SQLデータベースのクエリとデータ取得
- 天気API

これら2つの要素を組み合わせることで、Claudeにツール使用をさせることができます：

1. システムプロンプト - ツール使用の概念の説明と、アクセスできるツールの詳細な記述リストをClaudeに提供します
2. Claudeのツール使用リクエストを調整・実行するための制御ロジック

### ツール使用のロードマップ

*このレッスンでは現在のツール使用フォーマットを教えていますが、近い将来、以下を含むツール使用機能を更新・改善する予定です：*
* *関数定義と呼び出しのより合理化されたフォーマット*
* *より堅牢なエラー処理とエッジケースのカバレッジ*
* *APIの他の部分とのより緊密な統合*
* *特により複雑なツール使用タスクのための、より良い信頼性とパフォーマンス*

### 例

Claudeでツール使用を有効にするには、システムプロンプトから始めます。この特別なツール使用システムプロンプトで、Claudeに以下を伝えます：
* ツール使用の基本的な前提とその内容
* Claudeが与えられたツールをどのように呼び出して使用できるか
* この特定のシナリオでアクセスできるツールの詳細なリスト

以下は、Claudeにツール使用を説明するシステムプロンプトの最初の部分です。システムプロンプトのこの部分は、Claudeにツール使用を促すすべてのインスタンスで一般化できます。私たちがClaudeに与えているツール呼び出し構造（`<function_calls> [...] </function_calls>`）は、Claudeが使用するように特別に訓練された構造ですので、これに従うことをお勧めします。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

以下は、この特定の状況でClaudeがアクセスできる正確なツールを定義するシステムプロンプトの2番目の部分です。この例では、Claudeに電卓ツールを提供します。これは3つのパラメータを取ります：2つのオペランドと1つの演算子です。

その後、システムプロンプトの2つの部分を組み合わせます。

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

これで、`calculator`ツールの使用を必要とする質問をClaudeに与えることができます。`stop_sequences`で`<function_calls\>`を使用して、Claudeが関数を呼び出すかどうか、いつ呼び出すかを検出します。

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

次に、Claudeの関数呼び出しからパラメータを抽出し、実際にClaudeの代わりに関数を実行できます。

まず、関数のコードを定義します。

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

次に、Claudeの関数呼び出し応答からパラメータを抽出します。すべてのパラメータが存在する場合、電卓ツールを実行します。

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

結果が得られたので、その結果を適切にフォーマットして、Claudeに戻すときにClaudeがその結果がどのツールに関連しているかを理解できるようにする必要があります。Claudeが認識するよう訓練されたフォーマットが設定されています：
```
<function_results>
<result>
<tool_name>{TOOL_NAME}</tool_name>
<stdout>
{TOOL_RESULT}
</stdout>
</result>
</function_results>
```

以下のセルを実行して、上記のツール結果をこの構造にフォーマットしてください。

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

あとは、この結果を以前と同じメッセージチェーンに追加してClaudeに送り返すだけです！

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

ツール使用チェーン全体をエンドツーエンドで実行したことを祝福します！

では、与えられたツールを全く必要としない質問をClaudeに与えたらどうなるでしょうか？

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

成功です！ご覧のとおり、Claudeは必要ないときには関数を呼び出さないことを知っていました。

上記の内容を変更せずにレッスンのプロンプトを実験したい場合は、レッスンノートブックの最下部にある[**プレイグラウンド例**](#example-playground)をご覧ください。

---

## 演習
- [演習 10.2.1 - SQL](#exercise-1021---SQL)

### 演習 10.2.1 - SQL
この演習では、世界最小の「データベース」にクエリを実行し、書き込むためのツール使用プロンプトを作成します。以下は初期化されたデータベースです（実際には単なる辞書です）。

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

そして、データベースに書き込み、読み取りを行う関数のコードです。

In [ ]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

演習を解くには、上記の`system_prompt_tools_specific_tools`のようなシステムプロンプトを定義することから始めてください。各ツールの名前と説明、および各関数の各パラメータの名前、タイプ、説明を含めるようにしてください。以下に開始用の骨組みを用意しました。

In [ ]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

準備ができたら、以下の例でツール定義システムプロンプトを試すことができます。以下のセルを実行するだけです！

In [ ]:
examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Get & print Claude's response
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

正しく実行できていれば、関数呼び出しメッセージは`add_user`、`add_product`、`get_user`、`get_product`関数を正しく呼び出すはずです。

追加の課題として、コードセルを追加してパラメータ解析コードを書いてください。その後、Claudeが提供したパラメータで関数を呼び出して、呼び出し後の「データベース」の状態を確認してください。

❓ 解答例を見たい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_10_2_1_solution)

### おめでとうございます！

ツール使用と関数呼び出しを学んだことを祝福します！検索とRAGについてもっと学びたい場合は、最後の付録セクションに進んでください。

---

## プレイグラウンド例

このエリアでは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのような影響を与えるかを確認できます。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)